# GBDT(Gradient Boosting Decision Tree) 简介

## 背景知识

- 决策树

- 决策树构造原理
    - CART生成
    - 回归树：均方误差最小
    - 分类树：基尼系数
    
### 回归树
 
#### 回归树的函数表示
 
 $$f(x) = \sum_{m=1}^M c_mI(x \in R_m)$$
 
 M- 叶子节点数
 
 x 属于 Rm则 I为1， 否则为0
 
 #### 最优特征选取
 
 $$min_{j,s}[min_{c1} \sum_{x_1 \in Ri} (y_i - c_1)^2 + min_{c2} \sum_{x_1 \in R_2}(y_i - c_2)^2]$$
 
 j表示特征，s表示特征取值的临界分隔值
 
 $$R_1 = \{x|x^j <= s\},R_2 = \{x|x^j > s\}$$
 
 c1和c2分别代表R1和R2样本平均的label值
 
 #### 树的构建流程
 
 - 遍历所有特征，特征的最佳划分的应的得分，选取最小的分的特征
 
 - 将数据根据此选取的特征分化成两部分
 
 - 在左右两部分中遍历找到划分特征知道满足条件为止
 
### 分类树
 
#### 函数表示
 
与回归树一致
 
#### 最有特征选取
 
- 基尼系数

## 数学原理和构建方法

## XGBoost数学原理和构建方法